In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
from train_yh import *
from utils_yh import *

import argparse
import os
import gc
import time
import json
import random
import warnings
from pprint import pprint

import numpy as np
import pandas as pd
from einops import rearrange, reduce, repeat
import tez
import torch
import torch.nn as nn
from sklearn import metrics
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForMaskedLM

import transformers

import copy
import warnings
import os

import numpy as np
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoModelForMaskedLM
from joblib import Parallel, delayed 
from tez import enums
from tez.callbacks import Callback
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import re
import nltk

In [16]:
import os
from transformers import AutoTokenizer, AutoConfig
def download_hfmodel(model_name, download_path, cwd = os.getcwd()):
    '''download hfmodel to local
        Parameters
            - model_name : the name of hf model.
            - cwd : the path to download the model. the model will be saved at cwd/{name of model}.
    '''
    download_path = os.path.join(cwd, download_path)
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
    tokenizer.save_pretrained(download_path)
    
    config_model = AutoConfig.from_pretrained(model_name, output_hidden_states = True) 
    config_model.save_pretrained(download_path)

    backbone = AutoModelForMaskedLM.from_pretrained(model_name, config=config_model)
    backbone.save_pretrained(download_path)
    return

download_hfmodel('allenai/longformer-base-4096', 'mlm-longformer-base-4096')

Some weights of LongformerForMaskedLM were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
mlm = AutoModelForMaskedLM.from_pretrained(f'mlm-{args.model}')

In [9]:
tokenizer = AutoTokenizer.from_pretrained(f'mlm-{args.model}', add_prefix_space=True)

In [12]:
a = tokenizer(df['discourse_text'][0])

In [34]:
def foo():
    a = 10
    return _foo()

def _foo():
    return locals()['a']

In [35]:
foo()

KeyError: 'a'

In [15]:
a = mlm(input_ids = masked_input_ids)

In [30]:
if b is None:
    print('a')
    
else:
    print('b')

NameError: name 'b' is not defined

In [13]:
os.chdir('/home/enkeejunior1/kaggle_feedbackprize/fb-sbert-yh')

In [7]:
# setting
class set_args_submission:
    def __init__(self):
        self.model: str = 'longformer-base-4096'
        self.sbert: bool = False
        self.output: str = '../model'
        self.input: str = '../input'
        self.max_len: int = 4096
        self.valid_batch_size: int = 4
        self.kfold = 5
            
args = set_args_submission()

In [17]:
def back_translation(df, args, num_jobs):
    total_ids = df["id"].unique()
    total_ids_splits = np.array_split(total_ids, num_jobs)

    results = Parallel(n_jobs=num_jobs, backend="multiprocessing")(
        delayed(_back_translation_helper)(idx, args) for idx in total_ids_splits
    )
    
    return

def _back_translation_helper(total_ids, args):
    src2tgt_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
    src2tgt = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

    tgt2src_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
    tgt2src = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

    src2tgt.eval()
    tgt2src.eval()
    
    print('finish preparing the model')
    
    for i, idx in enumerate(total_ids):
        filename = os.path.join(args.input, "train", idx + ".txt")
        with open(filename, "r") as f:
            text = f.read()
        
        # translation
        input_ids_split = [src2tgt_tokenizer.encode(sent, return_tensors="pt") for sent in nltk.sent_tokenize(text)]
        outputs_split = []
        for input_ids in input_ids_split:
            outputs = src2tgt.generate(input_ids)
            outputs_split.append(outputs)
        decode_split = []
        for outputs in outputs_split:
            decode_split.append(src2tgt_tokenizer.decode(outputs.squeeze(), skip_special_tokens = True))
        
        # back-translation
        input_ids_split = [tgt2src_tokenizer.encode(sent, return_tensors="pt") for sent in decode_split]
        outputs_split = []
        for input_ids in input_ids_split:
            outputs = tgt2src.generate(input_ids)
            outputs_split.append(outputs)
        decode_split = []
        for outputs in outputs_split:
            decode_split.append(tgt2src_tokenizer.decode(outputs.squeeze(), skip_special_tokens = True))
        
        filename = os.path.join(args.input, "aug_back_translation", idx + ".txt")
        with open(filename, 'w') as f:
            f.write(' '.join(decode_split))
        
        if i % 10 == 0:
            print(f'current idx : {i}')
    return 

In [11]:
NUM_JOBS = 12

# read kfold data
if os.path.isfile(args.input + '/' + f'train_{args.kfold}folds.csv'):
    df = pd.read_csv(args.input + '/' + f'train_{args.kfold}folds.csv')
else:
    df = pd.read_csv(args.input_path + '/' + f'train.csv')
    df = get_stratified_fold(df, args)

144293

In [ ]:
back_translation(df, args, num_jobs = 20)

finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the modelfinish preparing the model

finish preparing the model
finish preparing the model
finish preparing the model
finish preparing the model
current idx : 0
current idx : 0
current idx : 0
current idx : 0
current idx : 10
current idx : 10
current idx : 10
current idx : 20
current idx : 10
current idx : 10


In [8]:

src2tgt_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
src2tgt = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

tgt2src_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
tgt2src = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

src2tgt.eval()
tgt2src.eval()

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(58101, 512, padding_idx=58100)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(58101, 512, padding_idx=58100)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [12]:
import psutil
n_jobs = psutil.cpu_count()

In [10]:
total_ids = df["id"].unique()
total_ids_splits = np.array_split(total_ids, 20)

In [ ]:
sys.ex